# Exercise 1.3 - Solution
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Importando pacote pandas para baixar os dados necessários
import pandas as pd

In [ ]:
# Pegando os dados
premierLeague2016_2017Raw = pd.read_csv('http://www.football-data.co.uk/mmz4281/1617/E0.csv')

# Analizando eles
premierLeague2016_2017Raw.head()

In [ ]:
# Limpando os dados com o que nos interessa
ourPLData = premierLeague2016_2017Raw[premierLeague2016_2017Raw.columns[1:6]]

# Renomeando algumas das colunas para algo mais legível
ourPLData.columns.values[3] = 'HomeTeamGoals'
ourPLData.columns.values[4] = 'AwayTeamGoals'

ourPLData

In [ ]:
# Vemos que a classificação dos dados da coluna Date não está no formato devido. Passaremos para tipo datetime quando estivermos gerando o dataframe final
ourPLData.dtypes

In [ ]:
# Criando o dataframe final
ourPLPivot = pd.DataFrame(columns=('Match', 'Date', 'Location', 'Team', 'Goals'))
for indx in range(0, len(ourPLData)):
    
    # Pegando dados de cada time por índice de partida
    homeTeam = []
    awayTeam = []
    homeTeam += [indx+1, pd.to_datetime(ourPLData[ourPLData.columns.values[0]].values[indx]), ourPLData[ourPLData.columns.values[1]].values[indx], ourPLData[ourPLData.columns.values[1]].values[indx], ourPLData[ourPLData.columns.values[3]].values[indx]]
    awayTeam += [indx+1, pd.to_datetime(ourPLData[ourPLData.columns.values[0]].values[indx]), ourPLData[ourPLData.columns.values[1]].values[indx], ourPLData[ourPLData.columns.values[2]].values[indx], ourPLData[ourPLData.columns.values[4]].values[indx]]
    
    # Passando os dados ao dataframe final através de temporário
    dataframe = pd.DataFrame([homeTeam, awayTeam], columns=('Match', 'Date', 'Location', 'Team', 'Goals'))
    ourPLPivot = ourPLPivot.append(dataframe)

# Ordenando tabela por ordem de partida primeiro, alfabética em segundo e finalizando arrumando os índices na tabela
ourPLPivot = ourPLPivot.sort_values(['Match','Team'], ascending=[True,True])
ourPLPivot = ourPLPivot.reset_index(drop = True)
ourPLPivot